In [1]:
import numpy as np
from models.nODE import nODE, make_nODE_from_parameters
import scipy.io

In [2]:
def distribute_parameters(param,adj):
    W1 = np.zeros((adj.shape[0],adj.shape[1]))
    k = 0
    for i in range(0, adj.shape[0]):
        for j in range(0, adj.shape[1]):
            if adj[i,j] > 0:
                W1[i,j] = param[k]
                k += 1

    gamma = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        gamma[i] = -param[k]
        k += 1

    b2 = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        b2[i] = param[k]
        k += 1
        
    b1 = np.zeros(adj.shape[0])
    for i in range(0, adj.shape[0]):
        b1[i] = param[k]
        k += 1

    W2 = np.zeros((adj.shape[0],adj.shape[1]))
    for i in range(0, adj.shape[0]):
        for j in range(0, adj.shape[1]):
            if i == j:
                W2[i,j] = param[k]
                k += 1
    
    return W1, gamma, b1, b2, W2


In [50]:
param1_true = np.loadtxt('data/dyn_bifurcating/fit_cluster0.txt')
param2_imputed = np.loadtxt('data/dyn_bifurcating/fit_cluster0_2_3.txt')

adj_imputed = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']
adj_imputed[2,3] = 0
adj_true = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']

(W1_1, gamma_1, b1_1, b2_1, W2_1) = distribute_parameters(param1_true,adj_true)
(W1_2, gamma_2, b1_2, b2_2, W2_2) = distribute_parameters(param2_imputed,adj_imputed)

# make_nODE_from_parameters(Gamma, Win=None, bin=None, Wout=None, bout=None)
nODE1_true = make_nODE_from_parameters(gamma_1, W1_1, b1_1, W2_1, b2_1)
nODE2_imputed = make_nODE_from_parameters(gamma_2, W1_2, b1_2, W2_2, b2_2)

In [60]:
from models.ranking import rank

# rank(nODE1,nODE2_base)

def network_grade(adjtrue, adjfit):
    missing_edge = [1 for i in range(adjtrue.shape[0]) for j in range(adjtrue.shape[1]) if adjtrue[i,j]!=0 and adjfit[i,j]== 0]
    # FOR DEBUGGING
    # print(missing_edge)
    number_missing_edges = len(missing_edge)
    ranking_network = 1 - number_missing_edges / np.max([1., np.count_nonzero(adjtrue)])
    ranking_network = max(ranking_network, 0)
    return ranking_network

# print(adj_true, adj_imputed)
print(network_grade(adj_true, adj_imputed))

0.9166666666666666


In [53]:
param2_gae = np.loadtxt('data/dyn_bifurcating/fit_cluster0_2_3_improved.txt')
param1_true = np.loadtxt('data/dyn_bifurcating/fit_cluster0.txt')

adj2_gae = scipy.io.loadmat('gae_results/dyn_bifurcating_2_3.mat')['inferred_adj']
adj2_gae = (adj2_gae>0.5).astype(int)
adj1_true = scipy.io.loadmat('data/dyn_bifurcating/net.mat')['Aref']

(W1_1, gamma_1, b1_1, b2_1, W2_1) = distribute_parameters(param1_true,adj1_true)
(W1_2, gamma_2, b1_2, b2_2, W2_2) = distribute_parameters(param2_gae,adj2_gae)

# make_nODE_from_parameters(Gamma, Win=None, bin=None, Wout=None, bout=None)
nODE1 = make_nODE_from_parameters(gamma_1, W1_1, b1_1, W2_1, b2_1)
nODE2_gae = make_nODE_from_parameters(gamma_2, W1_2, b1_2, W2_2, b2_2)

In [57]:
from models.ranking import rank

# rank(nODE1,nODE2_gae)
print(network_grade(adj1_true, adj2_gae))
print(adj1_true, '\n',adj2_gae)

0.08333333333333337
[[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 1 0 0]
 [1 0 0 1 1 1 1]
 [1 0 0 1 1 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] 
 [[0 0 1 0 0 0 0]
 [0 1 0 0 1 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 1 1]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0]]
